In [ ]:
import pandas as pd
from openai import OpenAI
from coordinate_constant import readfile


apikey:str = readfile('apikey.txt', '_r')
client = OpenAI(
    # This is the default and can be omitted
    api_key=apikey,
)

# Đọc file Excel và lấy dữ liệu từ cột 'title'
df = pd.read_excel('ds post.xlsx')

# Hàm kiểm tra từ nhạy cảm thông qua ChatGPT API
def check_sensitive_content(text):
    message_content = (
            f"Kiểm tra đoạn văn sau để xác định xem có chứa nội dung nhạy cảm không:\n\n'{text}'\n\n"
            "Các nội dung nhạy cảm cần kiểm tra bao gồm:\n"
            "- **Chính trị**: các từ liên quan đến bầu cử, chính quyền, phản động hoặc cách mạng.\n"
            "- **Bạo lực**: từ liên quan đến hành vi bạo lực, giết người, đánh đập, hoặc khủng bố.\n"
            "- **Khiêu dâm**: nội dung tình dục, khiêu dâm, đồi trụy hoặc nhạy cảm.\n\n"
            "Nếu đoạn văn có chứa bất kỳ loại nội dung nào trên, hãy trả lời 'Có' kèm theo loại nội dung. "
            "Nếu không, hãy trả lời 'Không'."
        )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": message_content}
        ],
        max_tokens=5,
        temperature=0
    )
    result = response.choices[0].message.content.strip()
    return result

# Duyệt qua các tiêu đề trong cột 'title' và xác nhận từ nhạy cảm
df['Sensitive Words'] = df['Title'].apply(check_sensitive_content)

# Xuất dữ liệu đã xử lý ra file Excel mới
df.to_excel('output_with_sensitive_words1.xlsx', index=False)

print("Đã kiểm tra từ nhạy cảm và lưu kết quả vào 'output_with_sensitive_words.xlsx'")
